In [1]:
from Bio import SeqIO

In [2]:
import os
import numpy as np

In [3]:
DATA_DIR = "./DNA_data_V3/Download"
TRAIN_DIR = DATA_DIR + "/Labeled"
TELOMERIC = TRAIN_DIR + "/telomeric"
NON_TELOMERIC = TRAIN_DIR + "/non-telomeric"
MIXED = TRAIN_DIR + "/mixed"
data_len = 250
TEST_DIR = DATA_DIR + "/Test"
TEST_CLASSIFICATION = TEST_DIR + "/test_tel_no-tel.fq"
TEST_MIXED = TEST_DIR + "/test_mixed"

In [4]:
telomerics = os.listdir(TELOMERIC)

In [5]:
non_telomerics = os.listdir(NON_TELOMERIC)

In [6]:
mixed = os.listdir(MIXED)

In [7]:
test_mixed = os.listdir(TEST_MIXED)

In [8]:
def let_to_num(let):
    if let == "A":
        return 0.1
    elif let == "T":
        return 0.2
    elif let == "G":
        return 0.3
    return 0.4

def read_dir(directory, files):
    data = []
    for f in files:
        gen = SeqIO.parse(directory + "/" + f, "fastq")
        for x in gen:
            seq = x.seq[:data_len]
            dp = np.zeros(data_len)
            for ind in range(len(seq)):
                dp[ind] = let_to_num(seq[ind])
            data.append(dp)
    return np.array(data)

In [9]:
def extract_length_from_filename(name):
    parts = name.split("@")[1].split(".")[0].split("_")
    return (int(parts[0].split('-')[1]) + int(parts[1].split('-')[1])) / 2

In [10]:
def extract_coverage_from_filename(name):
    return int(name.split("@")[0].split("_")[4].split("x")[0])

In [11]:
def extract_rl_from_filename(name):
    return int(name.split("@")[0].split("_")[3].split("bp")[0])

In [12]:
def read_mixed_file(f):
    data = []
    gen = SeqIO.parse(MIXED + "/" + f, "fastq")
    for x in gen:
        seq = x.seq[:data_len]
        dp = np.zeros(data_len)
        for ind in range(len(seq)):
            dp[ind] = let_to_num(seq[ind])
        data.append(dp)
    return np.array(data)

In [13]:
def read_mixed_file_test(f):
    data = []
    gen = SeqIO.parse(TEST_MIXED + "/" + f, "fastq")
    for x in gen:
        seq = x.seq[:data_len]
        dp = np.zeros(data_len)
        for ind in range(len(seq)):
            dp[ind] = let_to_num(seq[ind])
        data.append(dp)
    return np.array(data)

In [14]:
telomeric_data = read_dir(TELOMERIC, telomerics)

In [15]:
non_telomeric_data = read_dir(NON_TELOMERIC, non_telomerics)

In [16]:
telomeric_data.shape

(180000, 250)

In [17]:
non_telomeric_data.shape

(405545, 250)

In [18]:
tel_label = np.ones(telomeric_data.shape[0])
non_tel_label = np.zeros(non_telomeric_data.shape[0])

In [19]:
X = np.vstack((telomeric_data, non_telomeric_data))
Y = np.hstack((tel_label, non_tel_label))

In [20]:
# 0 = train, 1 = validation, 2 = test
random_index = np.random.choice([0, 1, 2], size=Y.size, p=[0.7, 0.3, 0])

In [21]:
#X_train = X[random_index == 0]
#Y_train = Y[random_index == 0]
X_val = X[random_index == 1]
Y_val = Y[random_index == 1]

In [22]:
#X_test = X[random_index == 2]
#Y_test = Y[random_index == 2]

In [23]:
#X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
#Y_train = Y_train.reshape((Y_train.shape[0], 1))
#X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
#Y_val = Y_val.reshape((Y_val.shape[0], 1))

In [24]:
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape

NameError: name 'X_train' is not defined

## Model

from keras import models
from keras.models import Sequential
from keras.layers import InputLayer, Dense, LSTM, Conv1D

model = Sequential()
model.add(InputLayer((1, data_len)))
model.add(Conv1D(5, 3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=100, batch_size=64, verbose=2)

X_train.shape

In [25]:
from multiprocessing import Pool
import pandas as pd

In [26]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [27]:
model = XGBClassifier(n_jobs=6, max_depth=4, n_estimators=150)
model.fit(X, Y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=150, n_jobs=6,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [28]:
pred = model.predict(X_val)

In [29]:
f1 = f1_score(Y_val, pred)
f1

0.9999169182198344

In [30]:
accuracy_score(Y_val, pred)

0.9999487652423404

In [41]:
mixed_lengths = [extract_length_from_filename(mix) for mix in mixed]

In [32]:
def get_telomere_length(mix):
    data = read_mixed_file(mix)
    pred = model.predict(data)
    print(pred.sum(), data.shape[0])
    coverage = extract_coverage_from_filename(mix)
    rl = extract_rl_from_filename(mix)
    return pred.sum() / coverage * rl / 2

In [42]:
pool = Pool(6)
lengths = pool.map(get_telomere_length, mixed)

706.0 23620
610.0 23560
644.0 23580
644.0 23580
644.0 23580
644.0 23600
706.0 23600
570.0 23490
506.0 23470
606.0 23580
605.0 23560
644.0 23580
643.0 23630
652.0 23580
661.0 23600
599.0 23540
734.0 23670
606.0 23580
516.0 23460
641.0 23630
580.0 81900
644.0 23580
706.0 23620
677.0 81930
585.0 23500
706.0 23620
581.0 81880
637.0 23580
584.0 81860
677.0 81930
643.0 23630
643.0 23630
573.0 81840
504.0 81800
706.0 23600
661.0 23600
504.0 23460
599.0 23540
540.0 81850
598.0 23550
581.0 81880
598.0 23550
504.0 81800
643.0 23630
584.0 81860
569.0 81850
645.0 81920
687.0 81960
585.0 23500
727.0 23670
584.0 81860
727.0 23670
706.0 23600
652.0 23580
570.0 23490
706.0 23620
658.0 23590
643.0 23630
506.0 23470
643.0 23630
628.0 23630
628.0 23630
644.0 23600
477.0 81750
570.0 23490
556.0 23510
477.0 81750
661.0 23600
599.0 23540
658.0 23590
628.0 23630
599.0 23540
632.0 81930
477.0 81750
599.0 23540
644.0 23580
556.0 23510
644.0 23580
652.0 23580
585.0 23500
727.0 23670
504.0 81800
727.0 23670
645.

In [43]:
mixed_lengths = np.array(mixed_lengths)
mixed_lengths

array([3147.5, 2580.5, 3121. , 2543.5, 3324.5, 2753. , 2506. , 2524. ,
       3324.5, 3121. , 3324.5, 3400. , 2299.5, 2813.5, 2753. , 3593.5,
       3121. , 2299.5, 2937.5, 3048.5, 2502.5, 3266.5, 3041.5, 3266.5,
       3121.5, 3190.5, 3324.5, 3121.5, 2931. , 2976. , 3103.5, 3593.5,
       3387.5, 3387.5, 3166. , 2813. , 3121. , 3387.5, 3266.5, 3068. ,
       2892. , 3387.5, 3324.5, 3121.5, 3262. , 3147.5, 3582.5, 3147.5,
       3190.5, 3387.5, 3266.5, 2960. , 2813. , 3593.5, 2678. , 2580.5,
       3262. , 2813.5, 2506. , 3266.5, 3121.5, 2506. , 2892. , 2761. ,
       3234. , 2761. , 3147.5, 3040.5, 2813. , 3387.5, 3234. , 2976. ,
       2813. , 2299.5, 3400. , 2960. , 3147.5, 2813. , 2976. , 3582.5,
       2892. , 2761. , 3103.5, 2976. , 3190.5, 3387.5, 3582.5, 3262. ,
       3324.5, 2678. , 3387.5, 2960. , 2931. , 2761. , 2751. , 2299.5,
       3234. , 3218.5, 3147.5, 3121. , 3593.5, 3387.5, 3166. , 2299.5,
       3266.5, 3121. , 3582.5, 3266.5, 3324.5, 3147.5, 3147.5, 3048.5,
      

In [44]:
lengths = np.array(lengths)
lengths

array([3220., 2530., 3260., 2580., 3530., 2925., 2520., 2520., 3435.,
       3260., 3530., 3140., 2385., 2780., 2925., 3635., 3260., 2385.,
       3005., 3025., 2675., 3530., 3050., 3530., 3030., 3385., 3530.,
       3185., 2905., 2990., 3225., 3635., 3215., 3215., 3160., 2920.,
       3260., 3215., 3530., 3015., 2890., 3215., 3530., 3185., 3220.,
       3220., 3670., 3220., 3385., 3215., 3530., 2995., 2920., 3635.,
       2850., 2530., 3220., 2780., 2520., 3530., 3185., 2520., 2890.,
       2700., 3305., 2700., 3220., 2900., 2920., 3215., 3305., 2990.,
       2920., 2385., 3140., 2995., 3220., 2920., 2990., 3670., 2890.,
       2700., 3225., 2990., 3385., 3215., 3670., 3220., 3530., 2850.,
       3215., 2995., 2905., 2700., 2845., 2385., 3305., 3290., 3220.,
       3260., 3635., 3215., 3160., 2385., 3530., 3260., 3670., 3530.,
       3435., 3220., 3220., 3025., 3305., 3030., 3205., 2865., 2520.,
       3215., 2925., 3530., 3290., 3140., 2850., 2995., 2995., 3225.,
       2890., 3260.,

In [45]:
for i in range(len(mixed_lengths)):
    print(extract_rl_from_filename(mixed[i]), mixed_lengths[i], lengths[i])
    rl = extract_rl_from_filename(mixed[i])
    cov = extract_coverage_from_filename(mixed[i])
    q1, q2 = (mixed_lengths[i] * 2 * cov / rl, lengths[i] * 2 * cov / rl)
    print(q1, q2, (q1 - q2) / q1)

100 3147.5 3220.0000000000005
629.5 644.0000000000001 -0.023034154090548233
100 2580.5 2530.0
516.1 506.0 0.019569850804107773
100 3121.0 3260.0
624.2 652.0 -0.0445370073694328
100 2543.5 2580.0
508.7 516.0 -0.014350304698250466
100 3324.5 3529.9999999999995
664.9 705.9999999999999 -0.061813806587456624
100 2753.0 2925.0
550.6 585.0 -0.06247729749364325
100 2506.0 2520.0
501.2 504.0 -0.0055865921787709725
100 2524.0 2520.0
504.8 504.0 0.0015847860538827482
100 3324.5 3435.0
664.9 687.0 -0.033238080914423256
100 3121.0 3260.0
624.2 652.0 -0.0445370073694328
100 3324.5 3529.9999999999995
664.9 705.9999999999999 -0.061813806587456624
100 3400.0 3140.0
680.0 628.0 0.07647058823529412
100 2299.5 2385.0
459.9 477.0 -0.03718199608610573
100 2813.5 2780.0
562.7 556.0 0.011906877554647316
100 2753.0 2925.0
550.6 585.0 -0.06247729749364325
100 3593.5 3635.0
718.7 727.0 -0.011548629469876101
100 3121.0 3260.0
624.2 652.0 -0.0445370073694328
100 2299.5 2385.0
459.9 477.0 -0.03718199608610573
100 2

100 2813.5 2780.0
562.7 556.0 0.011906877554647316
100 2506.0 2520.0
501.2 504.0 -0.0055865921787709725
100 3121.0 3260.0
624.2 652.0 -0.0445370073694328
100 3262.0 3220.0000000000005
652.4 644.0000000000001 0.012875536480686487
100 3593.5 3635.0
718.7 727.0 -0.011548629469876101
100 2937.5 3005.0
587.5 601.0 -0.02297872340425532
100 2892.0 2890.0
578.4 578.0 0.0006915629322267933
100 2960.0 2995.0
592.0 599.0 -0.011824324324324325
100 3218.5 3290.0
643.7 658.0 -0.022215317694578147
100 3121.5 3185.0
624.3 637.0 -0.02034278391798822
100 3121.0 3260.0
624.2 652.0 -0.0445370073694328
100 3161.0 3210.0
632.2 642.0 -0.01550142360012647
100 2753.0 2925.0
550.6 585.0 -0.06247729749364325
100 2937.5 3005.0
587.5 601.0 -0.02297872340425532
100 2813.0 2920.0
562.6 584.0 -0.038037682189832875
100 3396.0 3204.9999999999995
679.2 640.9999999999999 0.05624263839811566
100 3262.0 3220.0000000000005
652.4 644.0000000000001 0.012875536480686487
100 3121.5 3030.0
624.3 606.0 0.029312830370014347
100 32

In [46]:
mixed_lengths - lengths

array([ -72.5,   50.5, -139. ,  -36.5, -205.5, -172. ,  -14. ,    4. ,
       -110.5, -139. , -205.5,  260. ,  -85.5,   33.5, -172. ,  -41.5,
       -139. ,  -85.5,  -67.5,   23.5, -172.5, -263.5,   -8.5, -263.5,
         91.5, -194.5, -205.5,  -63.5,   26. ,  -14. , -121.5,  -41.5,
        172.5,  172.5,    6. , -107. , -139. ,  172.5, -263.5,   53. ,
          2. ,  172.5, -205.5,  -63.5,   42. ,  -72.5,  -87.5,  -72.5,
       -194.5,  172.5, -263.5,  -35. , -107. ,  -41.5, -172. ,   50.5,
         42. ,   33.5,  -14. , -263.5,  -63.5,  -14. ,    2. ,   61. ,
        -71. ,   61. ,  -72.5,  140.5, -107. ,  172.5,  -71. ,  -14. ,
       -107. ,  -85.5,  260. ,  -35. ,  -72.5, -107. ,  -14. ,  -87.5,
          2. ,   61. , -121.5,  -14. , -194.5,  172.5,  -87.5,   42. ,
       -205.5, -172. ,  172.5,  -35. ,   26. ,   61. ,  -94. ,  -85.5,
        -71. ,  -71.5,  -72.5, -139. ,  -41.5,  172.5,    6. ,  -85.5,
       -263.5, -139. ,  -87.5, -263.5, -110.5,  -72.5,  -72.5,   23.5,
      

In [47]:
r = 1 - np.square(lengths - mixed_lengths).sum() / np.square(lengths - lengths.mean()).sum()

In [48]:
r

0.840548359027563

In [49]:
2 * r * f1 / (r + f1)

0.9133326612177118

## Predict on test

### Classification

In [50]:
TEST_CLASSIFICATION

'./DNA_data_V3/Download/Test/test_tel_no-tel.fq'

In [51]:
def seq_to_data_point(seq):
    seq = x.seq[:data_len]
    dp = np.zeros(data_len)
    for ind in range(len(seq)):
        dp[ind] = let_to_num(seq[ind])
    return dp.reshape((1, data_len))

In [52]:
test_names = []
test_labels = []
test_gen = SeqIO.parse(TEST_CLASSIFICATION, "fastq")
for x in test_gen:
    test_names.append(x.name)
    p = model.predict(seq_to_data_point(x.seq))
    test_labels.append(p[0])

In [53]:
df = pd.DataFrame(data={"ID": test_names, "Label": test_labels}, columns=["ID", "Label"])

In [54]:
df.to_csv("test_tel_no-tel_Davit_Khechoyan_2.csv", index=False)

### Telomere length

In [55]:
def get_telomere_length_test(mix):
    data = read_mixed_file_test(mix)
    pred = model.predict(data)
    print(pred.sum(), data.shape[0])
    coverage = extract_coverage_from_filename(mix)
    rl = extract_rl_from_filename(mix)
    return mix, pred.sum() / coverage * rl / 2

In [56]:
pool = Pool(6)
test_mixed_data = pool.map(get_telomere_length_test, test_mixed)

628.0 23630
535.0 81800
642.0 81930
645.0 81920
668.0 81970
642.0 81930
573.0 81840
584.0 81860
573.0 81840
504.0 81800
535.0 81800
632.0 81930
504.0 81800
598.0 23550
668.0 81970
632.0 81930
668.0 81970
601.0 81880
581.0 81880
641.0 23630
581.0 81880
670.0 81970
583.0 81860
628.0 23630
644.0 81910
677.0 81930
642.0 81930
727.0 23670
727.0 23670
569.0 81850
598.0 23550
601.0 81880
573.0 81840
477.0 81750
477.0 81750
598.0 81870
687.0 81960
540.0 81850
540.0 81850
583.0 81860
580.0 81900
642.0 81930
535.0 81800
645.0 81920
727.0 23670
584.0 81840
540.0 81850
641.0 23630
535.0 81800
573.0 81840
584.0 81840
584.0 81860
584.0 81860
598.0 81870
645.0 81920
645.0 81920
642.0 81930
727.0 23670
504.0 81800
727.0 23670
601.0 81880
687.0 81960
670.0 81970
668.0 81970
477.0 81750
677.0 81930
580.0 81900
581.0 81880
632.0 81930
632.0 81930
628.0 23630
668.0 81970
565.0 81860
687.0 81960
677.0 81930
580.0 81900
581.0 81880
580.0 81900
632.0 81930
584.0 81840
687.0 81960
540.0 81850
584.0 81860
668.

In [57]:
test_file_names = [x[0] for x in test_mixed_data]
test_tel_lenghts = [x[1] for x in test_mixed_data]

In [58]:
dfmix = pd.DataFrame(data={"FileName": test_file_names, "Label": test_tel_lenghts}, columns=["FileName", "Label"])

In [59]:
dfmix.to_csv("test_mixed_Davit_Khechoyan_2.csv", index=False)

### Compare results

In [60]:
old_len = pd.read_csv("test_mixed_Davit_Khechoyan.csv")
old_cls = pd.read_csv("test_tel_no-tel_Davit_Khechoyan.csv")

In [61]:
np.abs(old_cls["Label"].values - test_labels).sum()

528.0

In [62]:
old_len["Label"].values - test_tel_lenghts

array([140., 140.,  90.,  95., 100., 140., 130., 150., 145.,  65., 100.,
       140., 140., 100.,  95.,  65., 130., 140.,  90., 150.,  90., 105.,
       150., 140., 145., 100., 140., 140.,  70.,  95.,  90., 140., 100.,
       105., 110., 160.,  70., 140., 140., 150., 140.,  70.,  70., 135.,
       145., 140.,  95.,  65., 105., 140.,  90., 100., 135., 140., 145.,
        70., 100., 160.,  90., 140.,  90., 110., 100.,  70., 140., 160.,
       110., 140.,  90.,  70., 140., 130., 140.,  90.,  65.,  90.,  70.,
       110.,  70.,  70., 140., 110., 140., 150., 135., 140., 100., 160.,
       110., 140., 135., 140.,  65., 160.,  55., 130., 140.,  90., 100.,
        65., 140.,  90., 110., 110.,  95., 100., 140.,  70., 140., 105.,
       140., 135., 110., 140., 110.,  55., 110., 160., 140., 100., 100.,
       105., 135., 160., 100.,  95., 105., 105., 145., 140., 100., 160.,
       140., 110., 140., 150., 160., 100., 105., 110., 100., 140., 100.,
       145.])